In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.parameter import Parameter
import torch.nn.functional as F
from tqdm import tqdm
import math
import random
import numpy as np
import pandas as pd
import torch.optim as optim

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
class FeatureRegression(nn.Module):
    def __init__(self, input_size):
        super(FeatureRegression, self).__init__()
        self.build(input_size)

    def build(self, input_size):
        self.W = Parameter(torch.Tensor(input_size, input_size))
        self.b = Parameter(torch.Tensor(input_size))

        m = torch.ones(input_size, input_size).cuda() - torch.eye(input_size, input_size).cuda()
        self.register_buffer('m', m)

        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.W.size(0))
        self.W.data.uniform_(-stdv, stdv)
        if self.b is not None:
            self.b.data.uniform_(-stdv, stdv)

    def forward(self, x):
        z_h = F.linear(x, self.W * Variable(self.m), self.b)
        return z_h

class TemporalDecay(nn.Module):
    def __init__(self, input_size, output_size, diag = False):
        super(TemporalDecay, self).__init__()
        self.diag = diag

        self.build(input_size, output_size)

    def build(self, input_size, output_size):
        self.W = Parameter(torch.Tensor(output_size, input_size)).cuda()
        self.b = Parameter(torch.Tensor(output_size)).cuda()
        self.relu = nn.ReLU(inplace=False)
        if self.diag == True:
            assert(input_size == output_size)
            m = torch.eye(input_size, input_size).cuda()
            self.register_buffer('m', m)

        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.W.size(0))
        self.W.data.uniform_(-stdv, stdv)
        if self.b is not None:
            self.b.data.uniform_(-stdv, stdv)

    def forward(self, d):
        gamma = self.relu(F.linear(d, self.W, self.b))
        gamma = torch.exp(-gamma)
        return gamma

In [3]:
# Generator 모델
class Generator(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Generator, self).__init__()

        self.temp_decay_h = TemporalDecay(input_size, output_size = hidden_size, diag = False)
        self.temp_decay_r = TemporalDecay(input_size, input_size, diag = True)
        
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.batch_norm = nn.BatchNorm1d(hidden_size)

        self.build()

    def build(self):
        self.output_layer = nn.Linear(self.hidden_size, self.input_size, bias=True)
        
        self.z_layer = FeatureRegression(self.input_size)
        self.beta_layer = nn.Linear(self.input_size * 2, self.input_size)
        self.grucell = nn.GRUCell(self.input_size * 2, self.hidden_size)
        self.hidden_dropout = nn.Dropout(p=0.5)
        

    def loss(self, hat, y, m):
        return torch.sum(torch.abs((y - hat)) * m) / (torch.sum(m) + 1e-5)

    
    def forward(self, input):
        rbfs = input[:,0,::]
        delta = input[:,1,::]
        masks = input[:,2,::]

        hid = torch.zeros((rbfs.size(0), self.hidden_size)).cuda()

        x_loss = 0.0
        imputations = []
        c_hat_list = []
        for i in range(rbfs.size(1)):

            r = rbfs[:,i,:]
            d = delta[:,i,:]
            m = masks[:,i,:]

            gamma_r = self.temp_decay_r(d)
            gamma_h = self.temp_decay_h(d)

            hid = hid * gamma_h
            
            x_hat = self.output_layer(hid)
            x_loss += torch.sum(torch.abs(r - x_hat) * m) / (torch.sum(m) + 1e-5)

            r_c = m * r + (1 - m) * x_hat

            r_hat = self.z_layer(r_c)

            x_loss += torch.sum(torch.abs(r - r_hat) * m) / (torch.sum(m) + 1e-5)

            beta_weight = torch.cat([gamma_r, m], dim = 1)
            beta = torch.sigmoid(self.beta_layer(beta_weight))

            c_hat = beta * r_hat + (1 - beta) * x_hat
            x_loss += torch.sum(torch.abs(r - c_hat) * m) / (torch.sum(m) + 1e-5)

            c_c = m * r + (1 - m) * c_hat

            gru_input = torch.cat([c_c, m], dim = 1)
            c_hat_list.append(c_hat.unsqueeze(1))
            imputations.append(c_c.unsqueeze(1))
            
            # GRU cell
            self.hidden_dropout(hid)
            hid = self.grucell(gru_input, hid)
            
        imputations = torch.cat(imputations, dim = 1)
        c_hat_list = torch.cat(c_hat_list, dim = 1)

        return  c_hat_list, imputations, x_loss / rbfs.size(1)

In [4]:
from torch.utils.data import Dataset, DataLoader

In [5]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
device = "cuda" if torch.cuda.is_available() else "cpu"

def make_deltas(masks):
    deltas = []
    for h in range(len(masks)):
        if h == 0:
            deltas.append([1 for _ in range(masks.shape[1])])
        else:
            deltas.append([1 for _ in range(masks.shape[1])] + (1-masks[h]) * deltas[-1])
    
    return list(deltas)


class MyDataset(Dataset):
    def __init__(self, dataset, q):
        self.data = dataset
        self.q = q

    def __len__(self):
        return self.data.shape[1] // self.q

    def __getitem__(self, index):
        return self.data[:,index * self.q : index * self.q + self.q,:]

def missing_data_rbf(df,rbf, batch_size):
    
    values = ((df - df.mean()) / df.std()).values
    shp = values.shape
    rbf_df = pd.read_csv("./RBFresult/" + rbf)
    masks = ~np.isnan(values)
    
    masks = masks.reshape(shp)

    deltas = np.array(make_deltas(masks))
    values = torch.nan_to_num(torch.from_numpy(values).to(torch.float32))
    masks = torch.from_numpy(masks).to(torch.float32)
    deltas = torch.from_numpy(deltas).to(torch.float32)
    rbf_x = torch.from_numpy(rbf_df.values).to(torch.float32)
    dataset = torch.cat([values.unsqueeze_(0), deltas.unsqueeze_(0), masks.unsqueeze_(0), rbf_x.unsqueeze_(0)], dim = 0)
    
    mydata  = MyDataset(dataset, batch_size)
    data = DataLoader(mydata, batch_size, shuffle=True)

    return data

def val_missing_data_rbf(df,rbf):
    
    values = ((df - df.mean()) / df.std()).values
    shp = values.shape
    rbf_df = pd.read_csv("./RBFresult/" + rbf)
    
    masks = ~np.isnan(values)
    
    masks = masks.reshape(shp)

    deltas = np.array(make_deltas(masks))
    values = torch.nan_to_num(torch.from_numpy(values).to(torch.float32))
    masks = torch.from_numpy(masks).to(torch.float32)
    deltas = torch.from_numpy(deltas).to(torch.float32)
    rbf_x = torch.from_numpy(rbf_df.values).to(torch.float32)
    dataset = torch.cat([values.unsqueeze_(0), deltas.unsqueeze_(0), masks.unsqueeze_(0), rbf_x.unsqueeze_(0)], dim = 0).unsqueeze_(0)

    return dataset

def eval_model(model, rbf, realpath, dfpath):
    
    df = pd.read_csv("./dataset/" + dfpath).drop(['datetime'], axis = 1)
    dataset = val_missing_data_rbf(df,rbf)
    dataset = dataset.to(device)

    real = pd.read_csv("./dataset/" + realpath).drop(['datetime'], axis = 1)
    real_scaler = (real - df.mean()) / df.std()

    df_scaler = ((df-df.mean()) / df.std()).values
    masks = ~np.isnan(df_scaler)
    masks = torch.from_numpy(masks).to(torch.float32)
    
    eval_masks = ~np.isnan(real_scaler.values)
    eval_masks = torch.from_numpy(eval_masks).to(torch.float32)

    test_masks = eval_masks - masks
    real_scaler = torch.nan_to_num(torch.from_numpy(real_scaler.values).to(torch.float32))
    
    model.eval()
    imputations, x_loss, c_hat_list = model(dataset)

    Nonscale_imputataion = pd.DataFrame(imputations[0].cpu().detach() , columns= df.columns)
    Nonscale_imputataion = (Nonscale_imputataion * df.std()) + df.mean()
    
    real = real.fillna(0)
    print("Scale MAE :", torch.sum(torch.abs(imputations[0].cpu().detach() - real_scaler) * test_masks) / torch.sum(test_masks))
    print("Scale MRE :", torch.sum(torch.abs(imputations[0].cpu().detach() - real_scaler) * test_masks) / torch.sum(torch.abs(real_scaler * test_masks)))

    print("Original MAE :", np.sum(np.abs((Nonscale_imputataion - real).values * test_masks.cpu().numpy())) / np.sum(test_masks.cpu().numpy()))
    print("Original MRE :", np.sum(np.abs((Nonscale_imputataion - real).values * test_masks.cpu().numpy())) / np.sum(np.abs(real.values * test_masks.cpu().numpy())))

    return Nonscale_imputataion

def eval_bi_model(model,rbf, realpath, dfpath):
    
    df = pd.read_csv("./dataset/" + dfpath).drop(['datetime'], axis = 1)
    dataset = val_missing_data_rbf(df,rbf, 1)
    dataset = dataset.to(device)

    real = pd.read_csv("./dataset/" + realpath).drop(['datetime'], axis = 1)
    real_scaler = (real - df.mean()) / df.std()

    df_scaler = ((df-df.mean()) / df.std()).values
    masks = ~np.isnan(df_scaler)
    masks = torch.from_numpy(masks).to(torch.float32)
    
    eval_masks = ~np.isnan(real_scaler.values)
    eval_masks = torch.from_numpy(eval_masks).to(torch.float32)

    test_masks = eval_masks - masks
    real_scaler = torch.nan_to_num(torch.from_numpy(real_scaler.values).to(torch.float32))
    
    model.eval()
    loss, x_loss, back_x_loss, loss_c, biimputataion = model(dataset)

    Nonscale_imputataion = pd.DataFrame(biimputataion[0].cpu().detach() , columns= df.columns)
    Nonscale_imputataion = (Nonscale_imputataion * df.std()) + df.mean()
    
    real = real.fillna(0)
    print("Scale MAE :", torch.sum(torch.abs(biimputataion[0].cpu().detach() - real_scaler) * test_masks) / torch.sum(test_masks))
    print("Scale MRE :", torch.sum(torch.abs(biimputataion[0].cpu().detach() - real_scaler) * test_masks) / torch.sum(torch.abs(real_scaler * test_masks)))

    print("Original MAE :", np.sum(np.abs((Nonscale_imputataion - real).values * test_masks.cpu().numpy())) / np.sum(test_masks.cpu().numpy()))
    print("Original MRE :", np.sum(np.abs((Nonscale_imputataion - real).values * test_masks.cpu().numpy())) / np.sum(np.abs(real.values * test_masks.cpu().numpy())))

    return Nonscale_imputataion

In [53]:
dfpath = 'pm25_missing.txt'
df = pd.read_csv("./dataset/"+dfpath).drop(["datetime"], axis = 1)
rbfpath = 'air_1000_0.05_time.csv'
batch_size = 64
dataset = missing_data_rbf(df, rbfpath, batch_size)

In [54]:
dataset = missing_data_rbf(df, rbfpath, batch_size)

In [6]:
class MGRU(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(MGRU, self).__init__()

        self.temp_decay_h = TemporalDecay(input_size, output_size = hidden_size, diag = False)
        self.temp_decay_x = TemporalDecay(input_size, input_size, diag = True)
        self.temp_decay_r = TemporalDecay(input_size, input_size, diag = True)
        
        self.hidden_size = hidden_size
        self.input_size = input_size

        self.build()

    def build(self):
        self.output_layer = nn.Linear(self.hidden_size, self.input_size, bias=True)
        
        self.z_layer = FeatureRegression(self.input_size)
        self.beta_layer = nn.Linear(self.input_size * 2, self.input_size)
        self.grucell = nn.GRUCell(self.input_size * 2, self.hidden_size)
        self.concat_lyaer = nn.Linear(self.input_size * 2, self.input_size)
        

    def loss(self, hat, y, m):
        return torch.sum(torch.abs((y - hat)) * m) / (torch.sum(m) + 1e-5)

    
    def forward(self, input):
        values = input[:,0,::]
        delta = input[:,1,::]
        masks = input[:,2,::]
        rbfs = input[:,3,::]

        hid = torch.zeros((values.size(0), self.hidden_size)).cuda()

        x_loss = 0.0
        imputations = []
        c_hat_list = []
        for i in range(values.size(1)):

            v = values[:,i,:]
            d = delta[:,i,:]
            m = masks[:,i,:]
            r = rbfs[:,i,:]

            gamma_x = self.temp_decay_x(d)
            gamma_h = self.temp_decay_h(d)
            
            hid = hid * gamma_h

            r_hat = self.temp_decay_r(r)
            
            x_hat = self.output_layer(hid)
            x_loss += torch.sum(torch.abs(v - x_hat) * m) / (torch.sum(m) + 1e-5)

            RG = torch.cat([x_hat, r_hat], dim = 1)
            concat_hat = self.concat_lyaer(RG)
            x_loss += torch.sum(torch.abs(v - concat_hat) * m) / (torch.sum(m) + 1e-5)

            x_c = m * v + (1 - m) * x_hat

            z_hat = self.z_layer(x_c)
            x_loss += torch.sum(torch.abs(v - z_hat) * m) / (torch.sum(m) + 1e-5)

            beta_weight = torch.cat([gamma_x, m], dim = 1)
            beta = torch.sigmoid(self.beta_layer(beta_weight))

            c_hat = beta * z_hat + (1 - beta) * x_hat
            x_loss += torch.sum(torch.abs(v - c_hat) * m) / (torch.sum(m) + 1e-5)

            c_c = m * v + (1 - m) * c_hat

            gru_input = torch.cat([c_c, m], dim = 1)
            imputations.append(c_c.unsqueeze(dim = 1))
            c_hat_list.append(c_hat.unsqueeze(1))
            
            # GRU cell
            hid = self.grucell(gru_input, hid)

        c_hat_list = torch.cat(c_hat_list, dim = 1)
        imputations = torch.cat(imputations, dim = 1)
        return imputations, x_loss, c_hat_list

In [26]:
def train_MGRU(model, lr, epochs, dataset, device):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    model.train()
    progress = tqdm(range(epochs))
    
    imputation_list = []
    loss_list = []
    c_hat_list2 = []
    model.to(device)

    for epoch in progress:
        batch_loss = 0.0
        for data in dataset:
            data = data.to(device)
            imputations, x_loss, c_hat_list = model(data)
        
            optimizer.zero_grad()
            x_loss.backward()
            optimizer.step()
            
            batch_loss += x_loss
        progress.set_description("loss: {}".format(batch_loss))

    return x_loss
    

In [24]:
dfpath = 'pm25_missing.txt'
df = pd.read_csv("./dataset/"+dfpath).drop(["datetime"], axis = 1)
rbfpath = 'air_1000_0.05_time.csv'
batch_size = 64
dataset = missing_data_rbf(df, rbfpath, batch_size)

In [27]:
model = MGRU(36, 64)
model.to(device)
loss_list = train_MGRU(model, 0.001, 1000, dataset, device)
Nonscale_imputataion = eval_model(model,rbfpath, "pm25_ground.csv", "pm25_missing.csv")

loss: 110.54060363769531: 100%|██████████| 1000/1000 [15:12<00:00,  1.10it/s]


Scale MAE : tensor(0.1815)
Scale MRE : tensor(0.2620)
Original MAE : 14.64047915524505
Original MRE : 0.20565254274388195


In [ ]:
Nonscale_imputataion = eval_model(model,"air_1000_0.05_time.csv", "pm25_ground.csv", "pm25_missing.csv")

Scale MAE : tensor(0.2011)
Scale MRE : tensor(0.2903)
Original MAE : 16.20589607710959
Original MRE : 0.22764171174730274


In [28]:
Nonscale_imputataion = eval_model(model,"air_1000_0.05_time.csv", "pm25_ground.csv", "pm25_missing.csv")

Scale MAE : tensor(0.1815)
Scale MRE : tensor(0.2620)
Original MAE : 14.64047915524505
Original MRE : 0.20565254274388195


In [ ]:
model = MGRU(36, 64)
model.to(device)
loss_list = train_MGRU(model, 0.0001, 1000, dataset, device)
Nonscale_imputataion = eval_model(model,"air_1000_0.05_time.csv", "pm25_ground.csv", "pm25_missing.csv")

loss: 161.7032012939453: 100%|██████████| 1000/1000 [13:48<00:00,  1.21it/s]


Scale MAE : tensor(0.1987)
Scale MRE : tensor(0.2869)
Original MAE : 16.07232064329993
Original MRE : 0.22576539832068707


In [ ]:
model = MGRU(36, 64)
model.to(device)
loss_list = train_MGRU(model, 0.001, 500, dataset, device)
Nonscale_imputataion = eval_model(model,"air_1000_0.05_time.csv", "pm25_ground.csv", "pm25_missing.csv")

loss: 110.17478942871094: 100%|██████████| 500/500 [07:06<00:00,  1.17it/s]


Scale MAE : tensor(0.1806)
Scale MRE : tensor(0.2608)
Original MAE : 14.601126131568007
Original MRE : 0.2050997569164492


In [ ]:
rbfdf = pd.read_csv("./RBFresult/air_20_8.0_drop_0.2.csv")

In [ ]:
rbfdf = (rbfdf - rbfdf.mean()) / rbfdf.std()

In [ ]:
rbfdf.to_csv("./RBFresult/air_20_8.0_scale.csv", index = False)

In [ ]:
rbfdf = pd.read_csv("./RBFresult/air_20_8.0_scale.csv")

In [ ]:
rbfdf

,001001,001002,001003,001004,001005,001006,001007,001008,001009,001010,...,001027,001028,001029,001030,001031,001032,001033,001034,001035,001036
0,0.833702,0.074850,0.347782,0.198880,0.178503,0.126357,0.182780,0.115592,0.094246,-0.043076,...,-0.053620,0.222963,0.748346,-0.311618,0.216050,0.458980,0.099035,-0.035186,-0.343594,-0.420327
1,0.884523,0.229706,0.530275,0.321159,0.199503,0.277548,0.320911,0.300777,0.255071,0.051002,...,0.137924,0.259383,0.751663,-0.233680,0.393282,0.513269,0.245915,0.040810,-0.204572,-0.305951
2,0.854783,0.307665,0.723294,0.406509,0.139408,0.383615,0.421295,0.514432,0.393861,0.173122,...,0.379266,0.376184,0.641818,-0.129991,0.522623,0.475514,0.332519,0.239367,-0.029407,-0.161838
3,0.797969,0.382020,0.837884,0.464174,0.124071,0.477686,0.530302,0.717413,0.529407,0.288889,...,0.672896,0.510132,0.564856,0.003552,0.674533,0.428275,0.443649,0.455692,0.183708,0.013497
4,0.680214,0.576192,0.800103,0.426261,0.231408,0.586270,0.669133,0.814170,0.649634,0.325653,...,1.017126,0.516252,0.569529,0.169826,0.824753,0.329915,0.607453,0.527583,0.433548,0.219047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8754,0.190555,-0.068424,-0.175941,-0.172720,-0.262101,-0.253210,-0.212252,-0.198784,-0.133938,-0.321118,...,0.178321,-0.296517,0.159499,-0.006098,0.162669,1.012385,0.570752,-0.349070,-0.310987,-0.580916
8755,0.145157,-0.140374,-0.166147,-0.169738,-0.211978,-0.234623,-0.205259,-0.168571,-0.112179,-0.292837,...,0.112672,-0.296055,0.109013,-0.013756,0.192919,1.060401,0.780759,-0.016766,-0.257222,-0.580590
8756,0.093035,-0.186031,-0.111841,-0.133577,-0.143286,-0.174276,-0.049471,-0.117937,-0.060297,-0.230917,...,0.055072,-0.295588,0.096265,0.042617,0.268784,1.174698,1.114924,0.572203,-0.168022,-0.580264
8757,0.043069,-0.138666,-0.043633,-0.115748,-0.132931,-0.144740,0.291237,-0.108921,-0.020099,-0.163647,...,0.003419,-0.295117,0.094204,0.133922,0.279773,1.406187,1.501001,1.218251,-0.055078,-0.579938


In [ ]:
dfpath = 'pm25_missing.txt'
df = pd.read_csv("./dataset/"+dfpath).drop(["datetime"], axis = 1)
rbfpath = 'air_20_8.0_scale.csv'
batch_size = 64
dataset = missing_data_rbf(df, rbfpath, batch_size)

In [ ]:
model = MGRU(36, 64)
model.to(device)
loss_list = train_MGRU(model, 0.001, 1000, dataset, device)
Nonscale_imputataion = eval_model(model, rbfpath, "pm25_ground.csv", "pm25_missing.csv")

loss: 113.62510681152344: 100%|██████████| 1000/1000 [13:40<00:00,  1.22it/s]


Scale MAE : tensor(0.1776)
Scale MRE : tensor(0.2564)
Original MAE : 14.322701748137124
Original MRE : 0.20118877273298785


In [ ]:
model = MGRU(36, 64)
model.to(device)
loss_list = train_MGRU(model, 0.001, 1000, dataset, device)
Nonscale_imputataion = eval_model(model, rbfpath, "pm25_ground.csv", "pm25_missing.csv")

loss: 97.78351593017578: 100%|██████████| 1000/1000 [13:38<00:00,  1.22it/s]


Scale MAE : tensor(0.1771)
Scale MRE : tensor(0.2556)
Original MAE : 14.275960990012232
Original MRE : 0.2005322125441967


In [19]:
class BiMGRU(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(BiMGRU, self).__init__()
        
        self.fmGRU = MGRU(input_size, hidden_size)
        self.bmGRU = MGRU(input_size, hidden_size)
    
    def get_consistency_loss(self, pred_f, pred_b):
        loss = torch.pow(pred_f - pred_b, 2.0).mean()
        return loss
    
    def backdirect_data(self, tensor_):
        if tensor_.dim() <= 1:
            return tensor_
    
        indices = range(tensor_.size()[2])[::-1]
        indices = Variable(torch.LongTensor(indices), requires_grad = False)

        if torch.cuda.is_available():
            indices = indices.cuda()

        return tensor_.index_select(2, indices)
    
    def backdirect_imputation(self, tensor_):
        if tensor_.dim() <= 1:
            return tensor_
    
        indices = range(tensor_.size()[1])[::-1]
        indices = Variable(torch.LongTensor(indices), requires_grad = False)

        if torch.cuda.is_available():
            indices = indices.cuda()

        return tensor_.index_select(1, indices)

    def forward(self, dataset):
        back_dataset = self.backdirect_data(dataset)

        x_imputataions, x_loss, C_hat = self.fmGRU(dataset)
        back_x_imputataions, back_x_loss, back_C_hat = self.bmGRU(back_dataset)

        loss_c = self.get_consistency_loss(x_imputataions, self.backdirect_imputation(back_x_imputataions))
        loss = x_loss + back_x_loss + loss_c

        biimputataion = (x_imputataions +  self.backdirect_imputation(back_x_imputataions)) / 2

        return  loss, x_loss, back_x_loss, loss_c, biimputataion

In [20]:
def train_BiMGRU(model, lr, epochs, dataset, device):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    model.train()
    progress = tqdm(range(epochs))
    
    imputation_list = []
    loss_list = []
    model.to(device)

    for epoch in progress:
        batch_loss = 0.0
        batch_f_loss = 0.0
        batch_b_loss = 0.0
        batch_c_loss = 0.0
        for data in dataset:
            data = data.to(device)
            loss, x_loss, back_x_loss, loss_c, biimputataion = model(data)
        
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            imputation_list.append(biimputataion)

            batch_loss += loss
            batch_f_loss += x_loss
            batch_c_loss += loss_c
            batch_b_loss += back_x_loss
            
        progress.set_description("loss: {}, f_MGRU loss : {}, b_MGRU loss : {}, consistency Loss : {}".format(batch_loss, batch_f_loss, batch_b_loss, batch_c_loss))
        loss_list.append(loss)

    return loss_list

In [21]:
def Bieval_model(model, rbf, realpath, dfpath):
    
    df = pd.read_csv("./dataset/" + dfpath).drop(['datetime'], axis = 1)
    dataset = val_missing_data_rbf(df,rbf)
    dataset = dataset.to(device)

    real = pd.read_csv("./dataset/" + realpath).drop(['datetime'], axis = 1)
    real_scaler = (real - df.mean()) / df.std()

    df_scaler = ((df-df.mean()) / df.std()).values
    masks = ~np.isnan(df_scaler)
    masks = torch.from_numpy(masks).to(torch.float32)
    
    eval_masks = ~np.isnan(real_scaler.values)
    eval_masks = torch.from_numpy(eval_masks).to(torch.float32)

    test_masks = eval_masks - masks
    real_scaler = torch.nan_to_num(torch.from_numpy(real_scaler.values).to(torch.float32))
    
    model.eval()
    loss, x_loss, back_x_loss, loss_c, biimputataion = model(dataset)

    Nonscale_imputataion = pd.DataFrame(biimputataion[0].cpu().detach() , columns= df.columns)
    Nonscale_imputataion = (Nonscale_imputataion * df.std()) + df.mean()
    
    real = real.fillna(0)
    print("Scale MAE :", torch.sum(torch.abs(biimputataion[0].cpu().detach() - real_scaler) * test_masks) / torch.sum(test_masks))
    print("Scale MRE :", torch.sum(torch.abs(biimputataion[0].cpu().detach() - real_scaler) * test_masks) / torch.sum(torch.abs(real_scaler * test_masks)))

    print("Original MAE :", np.sum(np.abs((Nonscale_imputataion - real).values * test_masks.cpu().numpy())) / np.sum(test_masks.cpu().numpy()))
    print("Original MRE :", np.sum(np.abs((Nonscale_imputataion - real).values * test_masks.cpu().numpy())) / np.sum(np.abs(real.values * test_masks.cpu().numpy())))

    return Nonscale_imputataion

In [22]:
dfpath = 'pm25_missing.txt'
df = pd.read_csv("./dataset/"+dfpath).drop(["datetime"], axis = 1)
rbfpath = 'air_1000_0.05_time.csv'
batch_size = 64
dataset = missing_data_rbf(df, rbfpath, batch_size)

In [23]:
bimodel = BiMGRU(36, 64)
bimodel.to(device)
biloss_list = train_BiMGRU(bimodel, 0.001, 600, dataset, device)
biNonscale_imputataion = Bieval_model(bimodel, rbfpath, "pm25_ground.csv", "pm25_missing.csv")

  0%|          | 0/600 [00:00<?, ?it/s]

loss: 233.22569274902344, f_MGRU loss : 116.21894836425781, b_MGRU loss : 116.94601440429688, consistency Loss : 0.060727380216121674: 100%|██████████| 600/600 [15:42<00:00,  1.57s/it]


Scale MAE : tensor(0.1779)
Scale MRE : tensor(0.2568)
Original MAE : 14.371936851229467
Original MRE : 0.201880370599139
